In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NV_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3773327405,NV,"3645 LOSEE RAOD, NORTH LAS VEGAS, NV",36.226438,-115.122367,3.200300e+14
1,7175937006,NV,"1050 South 21st St, SPARKS, NV",39.524918,-119.771390,3.203100e+14
2,7646338105,NV,"2080 N Rainbow Blvd, LAS VEGAS, NV",36.197270,-115.241970,3.200300e+14
3,3465167305,NV,"8020 South Rainbow Boulevard, Suite 100-530, L...",36.043033,-115.241853,3.200300e+14
4,5910497003,NV,"3225-A SOUTH RAINBOW RD SUITE 102, LAS VEGAS, NV",36.130811,-115.243396,3.200300e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3773327405,NV,"3645 LOSEE RAOD, NORTH LAS VEGAS, NV",36.226438,-115.122367,3.200300e+14
1,7175937006,NV,"1050 South 21st St, SPARKS, NV",39.524918,-119.771390,3.203100e+14
2,7646338105,NV,"2080 N Rainbow Blvd, LAS VEGAS, NV",36.197270,-115.241970,3.200300e+14
3,3465167305,NV,"8020 South Rainbow Boulevard, Suite 100-530, L...",36.043033,-115.241853,3.200300e+14
4,5910497003,NV,"3225-A SOUTH RAINBOW RD SUITE 102, LAS VEGAS, NV",36.130811,-115.243396,3.200300e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
58,4761727208,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,NaN
134,6985387200,NV,"580 Mount Rose Street, RENO, NV",39.509367,-119.813504,NaN
253,4739527200,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,NaN
420,4468727209,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,NaN
427,4428507208,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,NaN
457,4385308605,NV,"300 W Sydney Dr, Sparks, NV",33.283954,-97.888525,NaN
482,4516547209,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,NaN
487,6384107006,NV,"300 W SYDNEY DR, SPARKS, NV",33.283954,-97.888525,NaN
599,5020967207,NV,"2826 Montessouri St., LAS VEGAS, NV",36.138135,-115.247048,NaN
665,5738727204,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('32')] #NV
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/30 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

30


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10991/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
58,4761727208,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,[320030029392019]
134,6985387200,NV,"580 Mount Rose Street, RENO, NV",39.509367,-119.813504,[320310003011013]
253,4739527200,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,[320030029392019]
420,4468727209,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,[320030029392019]
427,4428507208,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,[320030029392019]
457,4385308605,NV,"300 W Sydney Dr, Sparks, NV",33.283954,-97.888525,[]
482,4516547209,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,[320030029392019]
487,6384107006,NV,"300 W SYDNEY DR, SPARKS, NV",33.283954,-97.888525,[]
599,5020967207,NV,"2826 Montessouri St., LAS VEGAS, NV",36.138135,-115.247048,[320030029392016]
665,5738727204,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,[320030029392019]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10991/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
58,4761727208,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,320030029392019
134,6985387200,NV,"580 Mount Rose Street, RENO, NV",39.509367,-119.813504,320310003011013
253,4739527200,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,320030029392019
420,4468727209,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,320030029392019
427,4428507208,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,320030029392019
457,4385308605,NV,"300 W Sydney Dr, Sparks, NV",33.283954,-97.888525,NaN
482,4516547209,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,320030029392019
487,6384107006,NV,"300 W SYDNEY DR, SPARKS, NV",33.283954,-97.888525,NaN
599,5020967207,NV,"2826 Montessouri St., LAS VEGAS, NV",36.138135,-115.247048,320030029392016
665,5738727204,NV,"2926 Montessouri St., LAS VEGAS, NV",36.136077,-115.247038,320030029392019


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
457,4385308605,NV,"300 W Sydney Dr, Sparks, NV",33.283954,-97.888525,NaN
487,6384107006,NV,"300 W SYDNEY DR, SPARKS, NV",33.283954,-97.888525,NaN
3338,3942107204,NV,"520 S. 4TH STREET 2ND FLOOR, LAS VEGAS, NV",42.338492,-93.081028,NaN
3436,4539507701,NV,"238 N BIVENS ROAD, MONROE, NV",42.338492,-93.081028,NaN
4326,6615868310,NV,"783 Cimarron Ct, Mesquite, NV",42.338492,-93.081028,NaN
5056,2754077108,NV,"626 S. 7TH STREET 0.0, LAS VEGAS, NV",42.338492,-93.081028,NaN
5508,2051407202,NV,"783 CIMARRON CT, MESQUITE, NV",42.338492,-93.081028,NaN
7289,4116867800,NV,"98 LAKE PKWY E, HENDERSON, NV",38.956822,-119.938883,NaN
7514,2351147100,NV,"720 S 7TH ST 3rd Floor, LAS VEGAS, NV",42.338492,-93.081028,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
58,4761727208,320030029392019
134,6985387200,320310003011013
253,4739527200,320030029392019
420,4468727209,320030029392019
427,4428507208,320030029392019
457,4385308605,NaN
482,4516547209,320030029392019
487,6384107006,NaN
599,5020967207,320030029392016
665,5738727204,320030029392019


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NV_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,595329,3773327405,NV,"3645 LOSEE RAOD, NORTH LAS VEGAS, NV",36.226438,-115.122367,3.200300e+14
1,595330,7175937006,NV,"1050 South 21st St, SPARKS, NV",39.524918,-119.771390,3.203100e+14
2,595331,7646338105,NV,"2080 N Rainbow Blvd, LAS VEGAS, NV",36.197270,-115.241970,3.200300e+14
3,595332,3465167305,NV,"8020 South Rainbow Boulevard, Suite 100-530, L...",36.043033,-115.241853,3.200300e+14
4,595333,5910497003,NV,"3225-A SOUTH RAINBOW RD SUITE 102, LAS VEGAS, NV",36.130811,-115.243396,3.200300e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,595329,3773327405,NV,"3645 LOSEE RAOD, NORTH LAS VEGAS, NV",36.226438,-115.122367,3.200300e+14,NaN
1,595330,7175937006,NV,"1050 South 21st St, SPARKS, NV",39.524918,-119.771390,3.203100e+14,NaN
2,595331,7646338105,NV,"2080 N Rainbow Blvd, LAS VEGAS, NV",36.197270,-115.241970,3.200300e+14,NaN
3,595332,3465167305,NV,"8020 South Rainbow Boulevard, Suite 100-530, L...",36.043033,-115.241853,3.200300e+14,NaN
4,595333,5910497003,NV,"3225-A SOUTH RAINBOW RD SUITE 102, LAS VEGAS, NV",36.130811,-115.243396,3.200300e+14,NaN
...,...,...,...,...,...,...,...,...
8249,603578,9365537309,NV,"9870 W RED COACH AVE, LAS VEGAS, NV",36.244274,-115.312190,3.200300e+14,NaN
8250,603579,9428908405,NV,"8076 Satin Carnation Ln, Las Vegas, NV",36.308456,-115.300100,3.200300e+14,NaN
8251,603580,9462568501,NV,"600 S Rock Blvd, Sparks, NV",39.538950,-119.765410,3.203100e+14,NaN
8252,603581,9631538407,NV,"120 Country Club Dr Ste 24L, Incline Village, NV",39.240430,-119.941250,3.203100e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,595329,3773327405,NV,"3645 LOSEE RAOD, NORTH LAS VEGAS, NV",36.226438,-115.122367,3.200300e+14,NaN,320030036261040.0
1,595330,7175937006,NV,"1050 South 21st St, SPARKS, NV",39.524918,-119.771390,3.203100e+14,NaN,320310031112040.0
2,595331,7646338105,NV,"2080 N Rainbow Blvd, LAS VEGAS, NV",36.197270,-115.241970,3.200300e+14,NaN,320030034191004.0
3,595332,3465167305,NV,"8020 South Rainbow Boulevard, Suite 100-530, L...",36.043033,-115.241853,3.200300e+14,NaN,320030029822005.0
4,595333,5910497003,NV,"3225-A SOUTH RAINBOW RD SUITE 102, LAS VEGAS, NV",36.130811,-115.243396,3.200300e+14,NaN,320030029392036.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3773327405,NV,"3645 LOSEE RAOD, NORTH LAS VEGAS, NV",36.226438,-115.122367,320030036261040.0
1,7175937006,NV,"1050 South 21st St, SPARKS, NV",39.524918,-119.771390,320310031112040.0
2,7646338105,NV,"2080 N Rainbow Blvd, LAS VEGAS, NV",36.197270,-115.241970,320030034191004.0
3,3465167305,NV,"8020 South Rainbow Boulevard, Suite 100-530, L...",36.043033,-115.241853,320030029822005.0
4,5910497003,NV,"3225-A SOUTH RAINBOW RD SUITE 102, LAS VEGAS, NV",36.130811,-115.243396,320030029392036.0
...,...,...,...,...,...,...
8249,9365537309,NV,"9870 W RED COACH AVE, LAS VEGAS, NV",36.244274,-115.312190,320030032401006.0
8250,9428908405,NV,"8076 Satin Carnation Ln, Las Vegas, NV",36.308456,-115.300100,320030032292002.0
8251,9462568501,NV,"600 S Rock Blvd, Sparks, NV",39.538950,-119.765410,320310019032009.0
8252,9631538407,NV,"120 Country Club Dr Ste 24L, Incline Village, NV",39.240430,-119.941250,320310033072007.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
457,4385308605,NV,"300 W Sydney Dr, Sparks, NV",33.283954,-97.888525,NaN
487,6384107006,NV,"300 W SYDNEY DR, SPARKS, NV",33.283954,-97.888525,NaN
3338,3942107204,NV,"520 S. 4TH STREET 2ND FLOOR, LAS VEGAS, NV",42.338492,-93.081028,NaN
3436,4539507701,NV,"238 N BIVENS ROAD, MONROE, NV",42.338492,-93.081028,NaN
4326,6615868310,NV,"783 Cimarron Ct, Mesquite, NV",42.338492,-93.081028,NaN
5056,2754077108,NV,"626 S. 7TH STREET 0.0, LAS VEGAS, NV",42.338492,-93.081028,NaN
5508,2051407202,NV,"783 CIMARRON CT, MESQUITE, NV",42.338492,-93.081028,NaN
7289,4116867800,NV,"98 LAKE PKWY E, HENDERSON, NV",38.956822,-119.938883,NaN
7514,2351147100,NV,"720 S 7TH ST 3rd Floor, LAS VEGAS, NV",42.338492,-93.081028,NaN


Convert to File

In [19]:
fips_NV = fips_merge_drop

In [20]:
fips_NV.to_csv("../../../data/state_data/geo/geo_fips/23/NV_fips_scraped.csv") 